In [1]:
import pandas as pd
import json
import pickle
import re

In [2]:
data_loc = "D:\\論文相關\\data\\"

In [3]:
#讀資料
text_file = "ChatExport_2021-10-23\\2021-12-28.json"
#text_json = json.load(data_loc+text_file)
old_text_file = data_loc+"sentence_seg_20200308-20210828_\\20200803-20210828.json"
text_loc = data_loc+"sentence_seg_20200308-20210828_\\"
df = pd.read_json(data_loc+text_file)

In [9]:
df

,name,type,id,messages
0,股癌 chat,private_supergroup,1301096229,"{'id': 746896, 'type': 'message', 'date': '202..."
1,股癌 chat,private_supergroup,1301096229,"{'id': 746897, 'type': 'message', 'date': '202..."
2,股癌 chat,private_supergroup,1301096229,"{'id': 746898, 'type': 'message', 'date': '202..."
3,股癌 chat,private_supergroup,1301096229,"{'id': 746899, 'type': 'message', 'date': '202..."
4,股癌 chat,private_supergroup,1301096229,"{'id': 746900, 'type': 'message', 'date': '202..."
...,...,...,...,...
1001249,股癌 chat,private_supergroup,1301096229,"{'id': 1775486, 'type': 'message', 'date': '20..."
1001250,股癌 chat,private_supergroup,1301096229,"{'id': 1775487, 'type': 'message', 'date': '20..."
1001251,股癌 chat,private_supergroup,1301096229,"{'id': 1775488, 'type': 'message', 'date': '20..."
1001252,股癌 chat,private_supergroup,1301096229,"{'id': 1775489, 'type': 'message', 'date': '20..."


In [10]:
t = text_loc+"oldTG_result.json"
nt = pd.read_json(t)

In [34]:
nt.messages.iloc[-1]

{'id': 890275,
 'type': 'message',
 'date': '2021-07-04T15:03:15',
 'from': '股值甦嵩',
 'from_id': 6000211276,
 'reply_to_message_id': 890274,
 'text': '等等下班來追看看'}

In [6]:
old_df = pd.read_json(old_text_file)

In [17]:
df.messages

0          {'id': 746896, 'type': 'message', 'date': '202...
1          {'id': 746897, 'type': 'message', 'date': '202...
2          {'id': 746898, 'type': 'message', 'date': '202...
3          {'id': 746899, 'type': 'message', 'date': '202...
4          {'id': 746900, 'type': 'message', 'date': '202...
                                 ...                        
1001249    {'id': 1775486, 'type': 'message', 'date': '20...
1001250    {'id': 1775487, 'type': 'message', 'date': '20...
1001251    {'id': 1775488, 'type': 'message', 'date': '20...
1001252    {'id': 1775489, 'type': 'message', 'date': '20...
1001253    {'id': 1775490, 'type': 'message', 'date': '20...
Name: messages, Length: 1001254, dtype: object

In [10]:
test_msg = old_df.loc[:,"messages"][0]

{'id': 116415,
 'type': 'message',
 'date': '2020-08-03T22:55:03',
 'from': 'RAY CHENG',
 'from_id': 'user1199460410',
 'text': '不過裕隆明天應該往上噴吧'}

In [20]:
test_msg = old_df.loc[:,"messages"][0]["date"][:10]
dt = pd.to_datetime(test_msg)

In [56]:
re.sub(r'[^\u4e00-\u9fa5]','','ABC...中')

'中'

In [63]:
#去掉非中文字
def filter_non_chinese(s):
    filted = re.sub(r'[^\u4e00-\u9fa5]','',s)
    
    return filted

#把資料集依min字數 切開並存成pickle
def make_text_dict(df, old_df,length = 3, text_in_middle = True):
    df_msg = old_df.loc[:,"messages"]
    text_by_date = {}
    def helper(df_msg, text_by_date, if_break = True):
        counter = 0
        for i in df_msg:
            
            #data_to_date_form = pd.to_datetime(date) #日期轉成dateime format
            date_and_time_with_close_time = pd.to_datetime(i["date"]) +  pd.Timedelta(hours = 11)#因為收盤後算隔天的 所以+11小時
            if text_in_middle:
                time_str = str(date_and_time_with_close_time)
                date = time_str[:10]
            else:
                data_to_date_form = pd.to_datetime(i["date"][:10])
                today_date_start_trade = data_to_date_form+pd.Timedelta(hours = 9)#當日開盤時間
                today_date_close_trade = data_to_date_form+pd.Timedelta(hours = 13) #當日收盤時間
                date_and_time = pd.to_datetime(i["date"])
                if date_and_time>=today_date_start_trade and date_and_time<=today_date_close_trade:
                    continue
                else:
                    time_str = str(date_and_time_with_close_time)
                    date = time_str[:10]
            
        
            if if_break and  date =="2021-06-01":
                print("break here", date)
                break
            if i["type"] == "message":
                #print(i["text"])
                user = i["from"]
                if  type(i["text"]) == str and len(i["text"])>=length:
                    cur_str = filter_non_chinese(i["text"])
                    if text_by_date.get(date)!= None:
                        if text_by_date.get(date).get(user)!=None:
                            if len(cur_str):
                                text_by_date[date][user].append(cur_str)
                        else:
                            if len(cur_str):
                                text_by_date[date][user] = []
                                text_by_date[date][user].append(cur_str)
                    else:
                        text_by_date[date] = {}
                        if len(cur_str):
                            text_by_date[date][user] = []
                            text_by_date[date][user].append(cur_str)
                elif type(i["text"]) == list:
                    #print(date, i["text"])
                    for item in i["text"]:
                        if type(item) == str and len(item)>=length:
                            cur_str = filter_non_chinese(item)
                            if text_by_date.get(date)!= None:
                                if text_by_date.get(date).get(user)!=None:
                                    if len(cur_str):
                                        text_by_date[date][user].append(cur_str)
                                else:
                                    if len(cur_str):
                                        text_by_date[date][user] = []
                                        text_by_date[date][user].append(cur_str)
                            else:
                                text_by_date[date] = {}
                                if len(cur_str):
                                    text_by_date[date][user] = []
                                    text_by_date[date][user].append(cur_str)
                counter +=1
                if counter%1000 == 0:
                    print(date)
        return text_by_date
    
    temp = helper(df_msg, text_by_date)

    df_msg = df.loc[:,"messages"]
    temp = helper(df_msg, temp, if_break=False)
                            
    return temp
                            

In [65]:
with_middel_text = make_text_dict(df, nt, 0)
#全部的文本的dataset
with open(data_loc+"text_loc\\"+f"text_all_bigger_than_{0}_with_middle.pickle", "wb") as f:
    pickle.dump(with_middel_text, f)
    


2020-04-16
2020-04-23
2020-04-25
2020-04-29
2020-05-01
2020-05-06
2020-05-08
2020-05-10
2020-05-12
2020-05-14
2020-05-15
2020-05-18
2020-05-20
2020-05-21
2020-05-23
2020-05-25
2020-05-26
2020-05-28
2020-05-30
2020-05-31
2020-06-02
2020-06-04
2020-06-05
2020-06-07
2020-06-08
2020-06-11
2020-06-12
2020-06-13
2020-06-14
2020-06-16
2020-06-18
2020-06-19
2020-06-20
2020-06-21
2020-06-23
2020-06-23
2020-06-25
2020-06-27
2020-06-28
2020-06-29
2020-06-30
2020-07-01
2020-07-02
2020-07-03
2020-07-03
2020-07-05
2020-07-07
2020-07-07
2020-07-08
2020-07-09
2020-07-10
2020-07-11
2020-07-13
2020-07-14
2020-07-15
2020-07-16
2020-07-17
2020-07-18
2020-07-20
2020-07-21
2020-07-21
2020-07-22
2020-07-23
2020-07-24
2020-07-25
2020-07-27
2020-07-28
2020-07-28
2020-07-29
2020-07-30
2020-07-31
2020-07-31
2020-08-01
2020-08-02
2020-08-04
2020-08-04
2020-08-05
2020-08-05
2020-08-05
2020-08-06
2020-08-06
2020-08-07
2020-08-07
2020-08-08
2020-08-10
2020-08-10
2020-08-11
2020-08-11
2020-08-12
2020-08-12
2020-08-13

2021-07-07
2021-07-07
2021-07-07
2021-07-07
2021-07-07
2021-07-07
2021-07-07
2021-07-08
2021-07-08
2021-07-08
2021-07-08
2021-07-08
2021-07-08
2021-07-09
2021-07-09
2021-07-09
2021-07-09
2021-07-09
2021-07-09
2021-07-09
2021-07-10
2021-07-10
2021-07-10
2021-07-11
2021-07-11
2021-07-11
2021-07-11
2021-07-12
2021-07-12
2021-07-12
2021-07-12
2021-07-12
2021-07-12
2021-07-13
2021-07-13
2021-07-13
2021-07-13
2021-07-13
2021-07-13
2021-07-13
2021-07-13
2021-07-13
2021-07-14
2021-07-14
2021-07-14
2021-07-14
2021-07-14
2021-07-14
2021-07-14
2021-07-14
2021-07-14
2021-07-15
2021-07-15
2021-07-15
2021-07-15
2021-07-15
2021-07-15
2021-07-15
2021-07-16
2021-07-16
2021-07-16
2021-07-16
2021-07-16
2021-07-16
2021-07-16
2021-07-16
2021-07-17
2021-07-17
2021-07-17
2021-07-17
2021-07-18
2021-07-18
2021-07-18
2021-07-19
2021-07-19
2021-07-19
2021-07-19
2021-07-19
2021-07-19
2021-07-20
2021-07-20
2021-07-20
2021-07-20
2021-07-20
2021-07-20
2021-07-20
2021-07-20
2021-07-21
2021-07-21
2021-07-21
2021-07-21

2021-12-02
2021-12-02
2021-12-02
2021-12-02
2021-12-03
2021-12-03
2021-12-03
2021-12-03
2021-12-04
2021-12-05
2021-12-05
2021-12-05
2021-12-06
2021-12-06
2021-12-06
2021-12-07
2021-12-07
2021-12-07
2021-12-07
2021-12-08
2021-12-08
2021-12-08
2021-12-08
2021-12-08
2021-12-08
2021-12-09
2021-12-09
2021-12-09
2021-12-09
2021-12-09
2021-12-10
2021-12-10
2021-12-10
2021-12-10
2021-12-10
2021-12-11
2021-12-11
2021-12-12
2021-12-12
2021-12-13
2021-12-13
2021-12-13
2021-12-13
2021-12-14
2021-12-14
2021-12-14
2021-12-14
2021-12-14
2021-12-15
2021-12-15
2021-12-15
2021-12-16
2021-12-16
2021-12-16
2021-12-16
2021-12-16
2021-12-16
2021-12-16
2021-12-17
2021-12-17
2021-12-17
2021-12-17
2021-12-17
2021-12-17
2021-12-18
2021-12-18
2021-12-18
2021-12-18
2021-12-18
2021-12-19
2021-12-19
2021-12-19
2021-12-19
2021-12-19
2021-12-19
2021-12-20
2021-12-20
2021-12-20
2021-12-20
2021-12-20
2021-12-21
2021-12-21
2021-12-21
2021-12-21
2021-12-21
2021-12-22
2021-12-22
2021-12-22
2021-12-23
2021-12-23
2021-12-23

In [66]:
with_middel_text["2020-06-01"]

{'MK H': ['希望這是他們面試的彩蛋三天後我收到信邀我去當隊長', '真的不要'],
 'Kyle Chou': ['前機長就是強', '看到最後一段就想到最後是不是沒買庫藏股'],
 'AA': ['是想說在未來企業越來越重視裡頭成分股可能會比較容易接到大公司的單'],
 'Shen Jessy': ['扣款日的前一天錢要到位錢定要到券商凌晨會發動扣款',
  '通姦除罪化了交易所還是不容許劈腿只是你的情況可能需要事後你來分享了'],
 'Wei L': ['總覺得策略指數毛毛的',
  '覺得高配息標的長時間看其實手續費那些滿高的會偏向投入配息率沒那麼高的標的真的有需要再賣股拿現金不過退休部位夠大可能也需要債券降低波動之類的'],
 '安床龍人': ['其實中國名義上是有個民主政黨小黨更多營造很開放的樣子給老百姓看'],
 'fio': ['被領導玩壞隨想我必須把這個發出來給大家樂一樂'],
 'Shawn': ['五毛都是不帶腦的'],
 'Halladay Chu': ['主委好猛喔我試了分鐘還搞不定就放棄了',
  '呱吉就是他的小隊員',
  '如果你也花小時才完成老婆肯定會翻臉',
  '你這樣害我聯想到裡面可是兩個男人耶',
  '對吼',
  '波克夏在這波美股大反彈中並沒有跟著彈上來請問大家怎麼看是波克夏策略過時了還是說趁現在撿便宜買一些呢大盤或幾乎都回來了但是還在下面',
  '我最近在看巴菲特寫給股東的信看了波克夏經營理念有點想當股東',
  '真太空',
  '要集的呱吉才有大概分鐘左右有聊呱吉要跟去紐約',
  '請問如果原本使用其他券商如凱基那要換到富果是得把股票都賣出嗎還是可以直接轉移呢會不會要很貴的手續費啊想要推薦我老婆用但我自己只有開美股對台灣的一竅不通謝謝'],
 'How Dare u': ['火箭震動船隻造成天線跟衛星失聯所以一般都會過兩三天才會有完整的影片出現'],
 '先生 水鏡': ['這個沒說錯啊祖國是指美國至於中國當然還需要加油啦',
  '不用再浪費錢了我是股市的老菜雞今年二月退休後才開始看股市前一陣子開完第二個證券戶後就很開心地兩邊都按了申購於是就浪費了四十元',
  '一邊告訴我重複申購取消資格另一邊告訴我沒有中籤',
  '雖然都是退休族可是每個人退休時的狀況不同很難一概而論吧有的人退休時有千萬身價

In [4]:
with_middle = pd.read_pickle(data_loc+"text_loc\\"+"text_all_bigger_than_0_with_middle.pickle")

In [28]:
#抓句子例子

from random import sample
import re 
text_sample = []
text_len_dict = {}
for k in range(3,7):
    text_len_dict[k] = [0, []]
random_pick = sample(text_example_list[0].keys(), 4)
for i in random_pick:
    cur_date = text_example_list[0][i]
    for m in cur_date:
        cur_text = re.sub("[^\u4e00-\u9fa5]+", "", m)
        if len(cur_text) in text_len_dict.keys() and text_len_dict[len(cur_text)][0]<10:
            text_len_dict[len(cur_text)][0]+=1
            text_len_dict[len(cur_text)][1].append(cur_text)


In [29]:
text_len_dict

{3: [10,
  ['推一個', '住別賣', '買起來', '哈哈哈', '好重喔', '和服女', '要錄影', '脖子勒', '有默契', '大家好']],
 4: [10,
  ['利多狂出',
   '感謝解答',
   '哦哦感謝',
   '請看置頂',
   '可憐韭菜',
   '查無此人',
   '達人懂買',
   '正人君子',
   '我就沒用',
   '看手相勒']],
 5: [10,
  ['這個有動畫',
   '為何想賣呢',
   '資產配置吧',
   '都在更新吧',
   '市場會測中',
   '勸過好幾次',
   '這算內線吧',
   '改才是真的',
   '一條龍可以',
   '幹勒真的有']],
 6: [10,
  ['我只想買幾隻',
   '購買頁面有標',
   '當然是電腦啊',
   '直接去問貨代',
   '你幹嘛牙起來',
   '是啦哪次不是',
   '哩奶鯊寢侵勒',
   '林媽媽快自首',
   '有沒有刺刺的',
   '月那天梭哈嗎']]}